In [70]:
import pandas as pd
import collections
import numpy as np
from datetime import datetime, timedelta

df=pd.read_csv('../raw data/Groceries_dataset.csv')
df['Date'] =  pd.to_datetime(df['Date'])
all_items=df['itemDescription'].unique()
all_customers=df['Member_number'].unique()
print(df.head())
print('\n')
print(df.info())

   Member_number       Date   itemDescription
0           1808 2015-07-21    tropical fruit
1           2552 2015-05-01        whole milk
2           2300 2015-09-19         pip fruit
3           1187 2015-12-12  other vegetables
4           3037 2015-01-02        whole milk


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Member_number    38765 non-null  int64         
 1   Date             38765 non-null  datetime64[ns]
 2   itemDescription  38765 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 908.7+ KB
None



Rearrange the dataframe as a multi-index on customer# and date.  This groups items that were purchased together in a single transaction with a '|' separator between items.


In [2]:
def cust_aggregator(series):
    item_str=''
    for item in series.values:
        item_str+=item+'|'
    return item_str

#The .sum() aggregator smashed all the strings together with no spaces.
#cust_aggregator separates them with a dividor symbol instead.
#This makes splitting easier.

df=df.groupby(['Member_number','Date']).agg({'itemDescription': [cust_aggregator]})
print(df.head(20))

                                                         itemDescription
                                                         cust_aggregator
Member_number Date                                                      
1000          2014-06-24                  whole milk|pastry|salty snack|
              2015-03-15  sausage|whole milk|semi-finished bread|yogurt|
              2015-05-27                        soda|pickled vegetables|
              2015-07-24                    canned beer|misc. beverages|
              2015-11-25                       sausage|hygiene articles|
1001          2014-07-02                  sausage|whole milk|rolls/buns|
              2014-12-12                                whole milk|soda|
              2015-01-20            frankfurter|soda|whipped/sour cream|
              2015-02-05                               frankfurter|curd|
              2015-04-14                               beef|white bread|
1002          2014-04-26                           


Create a list of items in all individual transactions.  These can be used to find correlations between items later.


In [48]:
purchases=[]
for member in df.index:
    for purchase in df.loc[member].values:
        purchases.append(purchase.split('|')[0:-1])
print(purchases[0:10])  #print 10 purchases

[['whole milk', 'pastry', 'salty snack'], ['sausage', 'whole milk', 'semi-finished bread', 'yogurt'], ['soda', 'pickled vegetables'], ['canned beer', 'misc. beverages'], ['sausage', 'hygiene articles'], ['sausage', 'whole milk', 'rolls/buns'], ['whole milk', 'soda'], ['frankfurter', 'soda', 'whipped/sour cream'], ['frankfurter', 'curd'], ['beef', 'white bread']]



Create a counter for each customer's purchased items.  Print off one customer's list of purchased goods.


In [61]:
customer_purchases={}
for member,data in df.groupby(level='Member_number'):
    data=data.droplevel(0)
    customer_items=[]
    for purchase in data['itemDescription'].values:
        for item in purchase[0].split('|')[0:-1]:
            customer_items.append(item)
    customer_purchases[member]=collections.Counter(customer_items)

print(customer_purchases[1000])

Counter({'whole milk': 2, 'sausage': 2, 'pastry': 1, 'salty snack': 1, 'semi-finished bread': 1, 'yogurt': 1, 'soda': 1, 'pickled vegetables': 1, 'canned beer': 1, 'misc. beverages': 1, 'hygiene articles': 1})



Print off the most commonly purchased items for each customer:


In [72]:
for customer in all_customers:
    print(customer, customer_purchases[customer].most_common(3))

1808 [('meat', 1), ('whole milk', 1), ('sugar', 1)]
2552 [('chocolate', 2), ('whole milk', 2), ('other vegetables', 1)]
2300 [('pork', 1), ('fruit/vegetable juice', 1), ('sausage', 1)]
1187 [('soda', 2), ('citrus fruit', 1), ('domestic eggs', 1)]
3037 [('whole milk', 1), ('other vegetables', 1), ('white bread', 1)]
4941 [('soda', 3), ('canned beer', 2), ('rolls/buns', 2)]
4501 [('other vegetables', 2), ('brown bread', 1), ('processed cheese', 1)]
3803 [('chicken', 2), ('butter', 1), ('skin care', 1)]
2762 [('whole milk', 2), ('pastry', 1), ('red/blush wine', 1)]
4119 [('whole milk', 2), ('cream cheese ', 1), ('frozen vegetables', 1)]
1340 [('whole milk', 4), ('rolls/buns', 2), ('pot plants', 1)]
2193 [('whole milk', 3), ('bottled beer', 2), ('pastry', 2)]
1997 [('soda', 2), ('frankfurter', 2), ('pip fruit', 2)]
4546 [('oil', 1), ('margarine', 1), ('other vegetables', 1)]
4736 [('other vegetables', 2), ('pastry', 1), ('seasonal products', 1)]
1959 [('root vegetables', 2), ('pastry', 2),

1025 [('coffee', 1), ('shopping bags', 1), ('ham', 1)]
1354 [('bottled beer', 2), ('red/blush wine', 1), ('cream cheese ', 1)]
2822 [('shopping bags', 2), ('salty snack', 2)]
1818 [('other vegetables', 1), ('bottled water', 1), ('beverages', 1)]
3082 [('whole milk', 4), ('tropical fruit', 3), ('brown bread', 2)]
4398 [('curd', 1), ('salty snack', 1), ('onions', 1)]
4332 [('domestic eggs', 1), ('bottled beer', 1), ('pork', 1)]
1357 [('brown bread', 2), ('root vegetables', 1), ('canned beer', 1)]
1159 [('citrus fruit', 2), ('light bulbs', 1), ('frozen vegetables', 1)]
1889 [('beef', 2), ('root vegetables', 2), ('whole milk', 1)]
2652 [('pastry', 2), ('ham', 1), ('waffles', 1)]
2736 [('whole milk', 3), ('soda', 3), ('tropical fruit', 1)]
2002 [('pork', 1), ('meat', 1), ('rolls/buns', 1)]
3046 [('sausage', 3), ('curd', 2), ('whole milk', 2)]
2284 [('pork', 2), ('frankfurter', 2), ('bottled water', 1)]
1479 [('pip fruit', 2), ('sausage', 2), ('whole milk', 1)]
1237 [('other vegetables', 3),

1180 [('frozen potato products', 1), ('whole milk', 1), ('citrus fruit', 1)]
3226 [('ham', 1), ('yogurt', 1), ('meat spreads', 1)]
1224 [('UHT-milk', 1), ('cooking chocolate', 1), ('sausage', 1)]
4086 [('pastry', 4), ('napkins', 2), ('semi-finished bread', 1)]
4963 [('other vegetables', 2), ('bottled beer', 1), ('canned fish', 1)]
2153 [('soda', 1), ('shopping bags', 1), ('domestic eggs', 1)]
1399 [('other vegetables', 3), ('condensed milk', 1), ('softener', 1)]
2478 [('citrus fruit', 1), ('soda', 1), ('napkins', 1)]
3367 [('sausage', 1), ('long life bakery product', 1)]
3616 [('misc. beverages', 1), ('cling film/bags', 1), ('curd', 1)]
1141 [('rolls/buns', 2), ('whole milk', 1), ('soda', 1)]
3213 [('soda', 1), ('chocolate marshmallow', 1), ('hard cheese', 1)]
2533 [('other vegetables', 1), ('frozen vegetables', 1), ('ham', 1)]
2878 [('whipped/sour cream', 1), ('pastry', 1), ('rolls/buns', 1)]
4980 [('meat', 1), ('domestic eggs', 1)]
3087 [('yogurt', 2), ('citrus fruit', 1), ('margarin